# For expressions

After completing this lesson, you should be able to:
  - Understand the relationship between for expressions and higher order functions
  - Describe the usage of for expressions


For expressions are syntactic sugar that simplifies the work of programming a multistage transformation

Let's take an example:


In [2]:
val myNums = 1 to 3
myNums.map(i => (1 to i).map(j => i * j))

myNums = Range(1, 2, 3)


Vector(Vector(1), Vector(2, 4), Vector(3, 6, 9))

The following two examples show the same but in Java and Python.


#### Java:
```java
import java.util.ArrayList;

class test {
    public static void main(String[] args) {
        ArrayList<Integer> lst = new ArrayList<>();
        lst.add(1);
        lst.add(2);
        lst.add(3);

        ArrayList<ArrayList<Integer>> result = new ArrayList<>();
        ArrayList<Integer> subLst;


        for(int s=0; s < lst.size(); s++) {
            subLst = new ArrayList<>();
            int sValue = lst.get(s);

            for(int t=0; t < s+1; t++) {
                int tValue = sValue * (t + 1);
                subLst.add(tValue);
            }
            result.add(subLst);
        }

        System.out.println(result);
    }
}
```

#### Python:
```python
lst = range(1, 4) # python range is not inclusive

result = []
for i in lst:
    sub_result = [j * i for j in range(1, i+1)]
    result.append(sub_result)

print(result)
```

Since, we want a collections of integers rather than a collection of collections, we start the processing with `flatMap` instead of `map`.

In [6]:
myNums.flatMap(i => (1 to i).map(j => i * j))

Vector(1, 2, 4, 3, 6, 9)

We can simplify the appearance of the code in the example above by using for-expressions. 

In [5]:
val myNums = (1 to 3).toVector
val result: Vector[Int] = for {
  i <- myNums
  j <- 1 to i
} yield i * j

myNums = Vector(1, 2, 3)
result = Vector(1, 2, 4, 3, 6, 9)


Vector(1, 2, 4, 3, 6, 9)

# Patterns Matching


Pattern matching is a powerful construct that improves on the familiar `switch`/`case` syntax (which is absent in Scala). It is best to start with a simple example:

```scala

val result: Int = 10

result match {
  case 1 => println("found 1")
  case someInteger: Int => println("found some value of type Int")
  // the compiler will issue a warning because this code is acutally impossible to reach. Any value of type Int will match the second condition
  case _ => println("found something that is not 1 or some other integer")
}
```

Matching is started with the keyword `match` which is followed by a block of case statements. The left-hand side of the `=>` in the `case` statement supports several varieties of syntax. The right-hand side can contain arbitrary code.

In the first `case` statement, we've checking if the `result` we're trying to find a match for is an integer 1. If it is not, the pattern matching moves on to the next `case` statement. In the following statement, if the value of `result` has a type of `Int`. If that is true, the value of `result` is stored in the name `someInteger`, this is known as a `type` pattern. If this pattern match is not successful, ie if `result` is not of type `Int`, the next case is evaluated. This last statement is a catch-all that will successfully match anything. When it matches, the string `"found something that is not 1 or some other integer"` will be printed to the screen.
We can wrap this pattern matching expressions into a function and experiment with it.

```scala
def isOneOrInteger(value: Int): Unit =
  value match {
    case 1 => println("found 1")
    case someInteger: Int => println("found some value of type Int")
    case _ => println("found something that is not 1 or some other integer")
  }

isOneOrInteger(1)
isOneOrInteger(22)
```

With this definition of the function the compiler will not actually allow us to pass anything other than integers as parameters to the function. In order to get around this, we have to relax the type of `value` to `Any`. Make this change and pass parameters of other types into the function. Observe the output.

Pattern matching is very useful when you want to act on specific values of some case class or a hierarchy of types that represents a set of alternatives (also known as an Algebraic Data Type).

As a reminder, an Algebraic Data Type is a way to encode the fact that a certain type is represented by a finite set of alternatives. For instance, in Scala the Boolean type contains two values, `true` and `false`. We could represent these as distinct types. This is shown below:

```scala
sealed trait MyBoolean {
  def value: Boolean
}
case class True(value: Boolean = true) extends MyBoolean
case class False(value: Boolean = false) extends MyBoolean
```

This the typical representation of an ADT in Scala. Closing the `trait` with `sealed` means that all of the definitions that are inherit from this trait are are defined in the same Scala file where this definition is located.


As another example, we could model the behavior of a light switch with an ADT. This is shown below:

```scala
sealed trait LightSwitch
case object On extends LightSwitch
case object Off extends LightSwitch
```

Since we've notified the compiler, by using `sealed`, that all of the subtypes of `LightSwitch` are in this file, it (the compiler) will verify that we have covered all of the cases when we pattern match on a value of type `LightSwitch`. Assign a value to `light` and observe the behavior.

```scala
sealed trait LightSwitch
case object On extends LightSwitch
case object Off extends LightSwitch

val light: LightSwitch = Off

def status(value: LightSwitch): Unit = 
  value match {
    case On => println("the switch is on")
    case Off => println("this switch is off")
  }

status(light)
```

We can write something similar to the above for our custom `Boolean` type. These values are little bit more complex in that they actually carry some values. However, using pattern matching we can de-compose or `unapply` case class into its components. Set value of `b` to one of the possible alternatives and observe the difference.

```scala
sealed trait MyBoolean {
  def value: Boolean
}

case class True(override val value: Boolean = true) extends MyBoolean
case class False(override val value: Boolean = false) extends MyBoolean


val b: MyBoolean = False()

def isItActuallyTrue(value: MyBoolean): Unit =
  value match {
    case True(true) => println("it is actually true!")
    case True(false) => println("it is pretending to be true!")
    case False(true) => println("it is pretending to be false, but is actually true!")
    case False(false) => println("it is really false!")
  }
```